# Install libraries

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 465 kB 71.4 MB/s 


In [2]:
from konlpy.tag import Okt
import os
import numpy as np
import pandas as pd
from datetime import datetime
import json
import re
from tqdm.notebook import tqdm

# Prepare data

예시 : 인기 제품 40개의 리뷰 

In [3]:
df_pop = pd.read_csv('popular_40.csv')
df_pop

,Unnamed: 0,product,rank,brand,maker,shop,userID,title,content,score
0,0,NEW 아레나-X 제로 게이밍 의자,1,제닉스,제닉스,NaN,maba****,우선 엄청 편하다 오래된의자 이제안녕임시디즈같은거 살까했는데 어차피 8할은 게임용의...,우선 엄청 편하다 오래된의자 이제안녕임<br>시디즈같은거 살까했는데 어차피 8할은 ...,5
1,1,NEW 아레나-X 제로 게이밍 의자,1,제닉스,제닉스,NaN,mars****,식탁의자로 컴퓨터 의자를 대신해서사용하고 있었는데 장시간 컴퓨터를 하면허리가 아파지...,식탁의자로 컴퓨터 의자를 대신해서<br><br>사용하고 있었는데 장시간 컴퓨터를 하...,4
2,2,NEW 아레나-X 제로 게이밍 의자,1,제닉스,제닉스,NaN,yong****,빠른배송으로 잘도착했습니다근데 몇가지 단점이라기보단 개선했으면 좋았겠다하는점이 눈에...,빠른배송으로 잘도착했습니다<br>근데 몇가지 단점이라기보단 개선했으면 좋았겠다하는점...,4
3,3,NEW 아레나-X 제로 게이밍 의자,1,제닉스,제닉스,NaN,urbe****,"지인분이 쓴다길래 저도 샀습니다초기 불량이 조금 있다는 얘기를 듣고 걱정했는데, 역...",지인분이 쓴다길래 저도 샀습니다<br>초기 불량이 조금 있다는 얘기를 듣고 걱정했는...,5
4,4,NEW 아레나-X 제로 게이밍 의자,1,제닉스,제닉스,NaN,ca******,왜 게이밍 의자 중에 베스트1인�,"왜 게이밍 의자 중에 베스트1인지 알겠네요.<br>굳이 게이밍용이 필요하다기보단, ...",5
...,...,...,...,...,...,...,...,...,...,...
43591,43591,ARENA TYPE 1 게이밍 의자,40,제닉스,제닉스,NaN,sun4****,빠른 배송과 손쉬운 설치 앉았을때 착좌감도 너무 좋네요 ㅎㅎ게임하기 너무 좋아요,빠른 배송과 손쉬운 설치 <br>앉았을때 착좌감도 너무 좋네요 ㅎㅎ<br>게임하기 ...,5
43592,43592,ARENA TYPE 1 게이밍 의자,40,제닉스,제닉스,NaN,lky7****,아버지 선물해드렸는데 무지 좋아하시네요ㅎ조립도 간편하고 편하다고하시네요~!,아버지 선물해드렸는데 무지 좋아하시네요ㅎ<br>조립도 간편하고 편하다고하시네요~!,5
43593,43593,ARENA TYPE 1 게이밍 의자,40,제닉스,제닉스,NaN,kist****,배송빠르고 실수없이 왔어요 감사합니다,배송빠르고 실수없이 왔어요 감사합니다,5
43594,43594,ARENA TYPE 1 게이밍 의자,40,제닉스,제닉스,NaN,mmdo****,배송빠르고 의자완전편안합니다 적극추천합니다~,배송빠르고 의자완전편안합니다 적극추천합니다~,5


In [4]:
df_pop['pos/neg'] = np.where(df_pop['score'] >= 4, 1, 0)
df_pop[['content','score','pos/neg']]

,content,score,pos/neg
0,우선 엄청 편하다 오래된의자 이제안녕임<br>시디즈같은거 살까했는데 어차피 8할은 ...,5,1
1,식탁의자로 컴퓨터 의자를 대신해서<br><br>사용하고 있었는데 장시간 컴퓨터를 하...,4,1
2,빠른배송으로 잘도착했습니다<br>근데 몇가지 단점이라기보단 개선했으면 좋았겠다하는점...,4,1
3,지인분이 쓴다길래 저도 샀습니다<br>초기 불량이 조금 있다는 얘기를 듣고 걱정했는...,5,1
4,"왜 게이밍 의자 중에 베스트1인지 알겠네요.<br>굳이 게이밍용이 필요하다기보단, ...",5,1
...,...,...,...
43591,빠른 배송과 손쉬운 설치 <br>앉았을때 착좌감도 너무 좋네요 ㅎㅎ<br>게임하기 ...,5,1
43592,아버지 선물해드렸는데 무지 좋아하시네요ㅎ<br>조립도 간편하고 편하다고하시네요~!,5,1
43593,배송빠르고 실수없이 왔어요 감사합니다,5,1
43594,배송빠르고 의자완전편안합니다 적극추천합니다~,5,1


In [5]:
pos_reviews = list(df_pop['content'][df_pop['pos/neg'] == 1])
# neg_reviews = list(df_pop['content'][df_pop['pos/neg'] == 0])
pos_reviews[:3]

['우선 엄청 편하다 오래된의자 이제안녕임<br>시디즈같은거 살까했는데 어차피 8할은 게임용의자로 앉을거같아서 처음으로 게이밍의자 너로정해따!<br>설명서가 잘 나와있어서 그냥 틀린그림찾기하듯이 매의눈으로 관찰하면 설치할수잇음+가렌의 힘도 필요함 아근데 설명서 그 물건사진으로 보여주면 더 좋을거같음 그림으로그려진거말고 기름칠같은거나 뭔가 나사부분조여지는거 볼때 그림도 알아보기쉽지만 그냥...사진같은게 더 이해가잘올듯<br>바퀴구멍을 낄때가 제일 고난의시간이었음 아무리 쑤셔넣고 돌려도 안들어가지는데 이럴때쓰라고 생존하고계신 남친을 이용했더니 잘넣음 (우쭐해하는모습 꼴보기싫엇지만 바퀴없는의자는 안되니까 넘어가줌)<br>의자뒤로 젖힐때 소리는 게임에집중하다가 들으면 딸꾹질할정도로 놀람-친구놀려줄때 사용해도될듯<br>푹신하고 편한데 단점은 진짜 무겁고 무거움<br>피방알바햇던적잇엇는데 그의자는 가벼웟는데 이건무거웟지만 내무게도 무거우니 날견디려면 이런 튼튼이가 맞을듯<br>첨으로 게이밍용 삿는데 정말 뿌듯함(돈쓴보람을느낀달까..헛소리하지말라는 잔소리들을수잇다)<br>아쉬운건 팔부분도 뭔가 좀 부드러웟으면 좋겟음 올리는기능도잇는데 그냥안쓸듯(이런거원하면 안마기를사야하나)<br>아그리고 여름에 좀 땀찰거같음 등부분이나 엉덩이부분 지금도 오래기대고잇으면 약간 답답함 통풍잘되는 대나무 걸어둬야할듯 대나무짱이다<br>롤열심히할게요 봇듀원딜구함 난 그 유명한 브론즈&hearts;',
 "식탁의자로 컴퓨터 의자를 대신해서<br><br>사용하고 있었는데 장시간 컴퓨터를 하면<br><br>허리가 아파지기 시작했습니다.<br><br>\u200b<br><br>그래서 큰맘 먹고 게이밍 의자를 구입했습니다!<br><br>누군가 그랬는데, 의자는 비싸게 구입해야 된다고...<br><br>의자로 유명한 브랜드 몇 곳을 둘러보다가<br><br>그래도 제닉스에서 구입하는 것이 좋을 것 같아서<br><br>제닉스 게이밍 의자를 구입했습니다.<br><br><br><br>예전에 컴퓨터 책상을 구입하면서

In [6]:
# 정규표현식을 사용한 전처리

pos_reviews_new = []
for pos_review in pos_reviews:
    pos_reviews_new.append(re.sub("[^가-힣0-9A-Za-z\?\!\.\,]", "", pos_review))
print(len(pos_reviews_new))

38310


# Tokenization

In [7]:
positive_reviews = []
for positive in pos_reviews_new:
    # Okt 형태소 분석기 사용
    okt = Okt()
    # stem : 통일화 여부 
    # 품사 태깅 (pos)
    positive_reviews.append(okt.pos((positive), stem= True))

In [10]:
# 첫번째 리뷰의 품사태깅 결과 

positive_reviews[0]

[('우선', 'Noun'),
 ('엄청', 'Adverb'),
 ('편하다', 'Adjective'),
 ('오래되다', 'Adjective'),
 ('의자', 'Noun'),
 ('이제', 'Noun'),
 ('안녕', 'Noun'),
 ('임', 'Noun'),
 ('br', 'Alpha'),
 ('시디즈', 'Noun'),
 ('같다', 'Adjective'),
 ('살다', 'Verb'),
 ('하다', 'Verb'),
 ('어차피', 'Noun'),
 ('8', 'Number'),
 ('하다', 'Verb'),
 ('게임', 'Noun'),
 ('용의자', 'Noun'),
 ('로', 'Josa'),
 ('앉다', 'Verb'),
 ('같다', 'Adjective'),
 ('처음', 'Noun'),
 ('으로', 'Josa'),
 ('게이', 'Noun'),
 ('밍', 'Noun'),
 ('의자', 'Noun'),
 ('너', 'Noun'),
 ('로', 'Josa'),
 ('정해', 'Noun'),
 ('따다', 'Verb'),
 ('!', 'Punctuation'),
 ('br', 'Alpha'),
 ('설명', 'Noun'),
 ('서', 'Josa'),
 ('가다', 'Verb'),
 ('잘나오다', 'Verb'),
 ('그냥', 'Noun'),
 ('틀리다', 'Verb'),
 ('그림', 'Noun'),
 ('찾기', 'Noun'),
 ('하', 'Suffix'),
 ('듯이', 'Noun'),
 ('매', 'Noun'),
 ('의', 'Josa'),
 ('눈', 'Noun'),
 ('으로', 'Josa'),
 ('관찰', 'Noun'),
 ('하', 'Suffix'),
 ('면', 'Josa'),
 ('설치', 'Noun'),
 ('하다', 'Verb'),
 ('잇다', 'Verb'),
 ('가렌', 'Noun'),
 ('의', 'Josa'),
 ('힘', 'Noun'),
 ('도', 'Josa'),
 ('필요하다', 'Adjectiv

In [12]:
# 불용어사전 

stopwords_pos = pd.read_table('stopwords of positive reviews.txt', sep = '\n', header = None)
stopwords_pos

,0
0,의자
1,있다
2,이다
3,게이
4,아니다
...,...
983,고고
984,급니
985,핑쿠핑쿠
986,엄니


In [13]:
# 토큰의 길이가 2개 이상이고, 토큰이 stopwords 안에 없을때만 출력 => 불용어 아닌 것 출력
all_pos_words = []

for i in positive_reviews:
        pos_words =[]
        for token, pos in i:
            if (pos == 'Noun' or pos == 'Adjective') and len(token) >= 2 and not token in stopwords_pos:
                pos_words.append(token)
        all_pos_words.append(pos_words)
len(all_pos_words)

38310

In [18]:
# 불용어 제거 후, 4230번째 리뷰 토큰 확인

all_pos_words[4230]

['의자',
 '바퀴',
 '부드럽다',
 '의자',
 '자세',
 '이동해',
 '부드럽다',
 '허리',
 '쿠션',
 '헤드',
 '쿠션',
 '있다',
 '자세',
 '편하다',
 '일부러',
 '받침',
 '용하다',
 '받침',
 '불편하다',
 '제거',
 '사용']

# 리뷰 내 부정어가 섞여 있는 경우, 해당 리뷰 삭제

In [19]:
# 부정어 사전

neg_words_lst = pd.read_table('negative words.txt', sep = '\n', header = None)
neg_words_lst

,0
0,"어렵다,"
1,"힘들다,"
2,"불편하다,"
3,"아쉽다,"
4,"냄새,"
...,...
262,"민망하다,"
263,"축축하다,"
264,"보풀,"
265,"뿌셔져,"


In [ ]:
neg = []
for i in all_pos_words:
    for j in i:
        if j in neg_words_lst:
            neg.append(i)
            break
len(neg)

In [ ]:
for i in neg :
    all_pos_words.remove(i)
len(all_pos_words)

23174

In [ ]:
# 순수 긍정 내용이 담긴 리뷰 키워드 

pos_keywords = []
for i in all_pos_words:
    pos_keywords.extend(i)
pos_keywords

In [ ]:
from collections import Counter

c = Counter(pos_keywords)
c[:10]

Counter({'체격': 23,
         '매우': 549,
         '편이': 112,
         '교체': 45,
         '기기': 2,
         '상당하다': 128,
         '신중하다': 10,
         '비슷하다': 65,
         '체중': 18,
         '친구': 141,
         '정도': 701,
         '사용': 1760,
         '구성': 150,
         '괜찮다': 1009,
         '제품': 1646,
         '구매': 1630,
         '오후': 23,
         '배송': 3532,
         '빨랏구': 3,
         '바로': 275,
         '완성': 121,
         '생각': 1905,
         '작다': 212,
         '같다': 2759,
         '흠짓': 1,
         '높이': 207,
         '조절': 413,
         '안락하다': 84,
         '검정색': 13,
         '가죽': 257,
         '부분': 629,
         '솔직하다': 37,
         '기대': 71,
         '느낌': 555,
         '실물': 78,
         '고급스럽다': 117,
         '예정': 59,
         '전체': 137,
         '평가': 13,
         '가격': 2025,
         '대비': 1010,
         '기사': 49,
         '다음': 216,
         '일찍이': 1,
         '직딩': 1,
         '주말': 29,
         '조립': 4997,
         '정리': 13,
         '편하다': 5287,
         '일찍': 13

In [ ]:
from collections import Counter
c = Counter(pos_keywords)
# positive_count = c.most_common() # 상위 10개 출력하기

dic_pos_a = {'키워드':[], 
       '빈도':[]}
for i in c:
    dic_pos_a['키워드'].append(i[0])
    dic_pos_a['빈도'].append(i[1])
dic_pos_a

df_pos_a = pd.DataFrame(dic_pos_a)
df_pos_a

df_pos_a.to_csv('A_positive_keywords.csv', encoding='cp949')